# Damiano Bressanin 138075

CNN per classificazione binaria cani-gatti con 3 blocchi costituiti da: convoluzioni + ReLU e maxpooling.

Ho preso spunto da "How to Classify Photos of Dogs and Cats" di Jason Brownlee.

Fonte: https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/


# Test:
- Dataset 80-10-10;
- Foto RGB;
- Sì Data Augmentation;
- Rete più profonda

In [1]:
import os
import shutil
import random
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import numpy as np

from matplotlib.image import imread
from matplotlib import pyplot

random.seed(1)

2023-09-03 13:31:50.422723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Normalizzazione e Data Augmentation

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
#    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.8,1.2],
#    rotation_range=20,
#    width_shift_range=0.15,
#    height_shift_range=0.15,
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64

train_it = train_datagen.flow_from_directory("dataset_diviso_v2/train/",
                                             class_mode='binary',
                                             batch_size=batch_size,
                                             target_size=(200, 200),
                                             color_mode="rgb",
                                             shuffle=True,
                                             seed=42
                                            )

val_it = val_datagen.flow_from_directory('dataset_diviso_v2/valid/',
                                         class_mode='binary',
                                         batch_size=batch_size,
                                         target_size=(200, 200),
                                         color_mode="rgb",
                                         shuffle=True,
                                         seed=42)

test_it = test_datagen.flow_from_directory('dataset_diviso_v2/test/',
                                           class_mode='binary',
                                           batch_size=batch_size,
                                           target_size=(200, 200),
                                           color_mode="rgb",
                                           shuffle=True,
                                           seed=42)



Found 19997 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


# Creazione del modello:

In [3]:
def define_model(optimizer):

    model=Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.4))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [5]:
import datetime

unique_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + unique_name

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='epoch')

In [6]:
from tensorflow.keras.optimizers import SGD, Adam
model=define_model('adam')
#model=define_model(SGD(learning_rate=0.001, momentum=0.9))



history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=val_it, validation_steps=len(val_it), epochs=500, verbose=1, callbacks=[tensorboard_callback,early_stop],validation_split=0)

2023-09-03 13:31:55.074871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-03 13:31:55.081118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-03 13:31:55.081562: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-03 13:31:55.083912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-03 13:31:55.084383: I tensorflow/compile

Epoch 1/500


2023-09-03 13:31:58.616731: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-09-03 13:32:00.143459: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-03 13:32:02.770391: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1b016a00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-03 13:32:02.770442: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-09-03 13:32:02.776562: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-03 13:32:02.919430: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled

 60/313 [====>.........................] - ETA: 2:55 - loss: 5.5022 - accuracy: 0.5240

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:866: UserWarning: Truncated File Read
  warnings.warn(str(msg))


...

313/313 [==============================] - 228s 726ms/step - loss: 0.6822 - accuracy: 0.5612 - val_loss: 0.6636 - val_accuracy: 0.6112
Epoch 4/500
313/313 [==============================] - ETA: 0s - loss: 0.6583 - accuracy: 0.6111

2023-09-03 13:47:27.044511: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4915200000 exceeds 10% of free system memory.


313/313 [==============================] - 234s 747ms/step - loss: 0.6583 - accuracy: 0.6111 - val_loss: 0.6305 - val_accuracy: 0.6476
Epoch 5/500
313/313 [==============================] - ETA: 0s - loss: 0.5855 - accuracy: 0.6934

2023-09-03 13:51:23.151678: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4915200000 exceeds 10% of free system memory.


313/313 [==============================] - 236s 753ms/step - loss: 0.5855 - accuracy: 0.6934 - val_loss: 0.5084 - val_accuracy: 0.7444
Epoch 6/500
313/313 [==============================] - 240s 765ms/step - loss: 0.5368 - accuracy: 0.7340 - val_loss: 0.4923 - val_accuracy: 0.7712
Epoch 7/500
313/313 [==============================] - 244s 777ms/step - loss: 0.4927 - accuracy: 0.7658 - val_loss: 0.4347 - val_accuracy: 0.8044
Epoch 8/500
313/313 [==============================] - 248s 791ms/step - loss: 0.4435 - accuracy: 0.7925 - val_loss: 0.3907 - val_accuracy: 0.8276
Epoch 9/500
313/313 [==============================] - 237s 755ms/step - loss: 0.4115 - accuracy: 0.8148 - val_loss: 0.3710 - val_accuracy: 0.8340
Epoch 10/500
313/313 [==============================] - 237s 755ms/step - loss: 0.3770 - accuracy: 0.8313 - val_loss: 0.3639 - val_accuracy: 0.8396
Epoch 11/500
313/313 [==============================] - 245s 781ms/step - loss: 0.3387 - accuracy: 0.8522 - val_loss: 0.3431 - va

In [7]:
# valuto il modello

_, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
print('> %.3f' % (acc * 100.0))

40/40 [==============================] - 4s 105ms/step - loss: 0.2490 - accuracy: 0.9040
> 90.400


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 100, 100, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 64)      36928     
                                                        

In [9]:
model_dir = "modelli/"+ unique_name +".keras"
model.save(model_dir)

In [10]:
def summarize_diagnostics(history):
    pyplot.figure(figsize=(20, 15))
    
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='orange', label='train')
    pyplot.plot(history.history['val_loss'], color='blue', label='val')
    pyplot.legend(loc='upper right') # legenda
    pyplot.grid(which='both', linestyle='-', linewidth=1, color='black')  # griglia nera
    
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='red', label='train')
    pyplot.plot(history.history['val_accuracy'], color='green', label='val')
    pyplot.legend(loc='lower right')
    pyplot.grid(which='both', linestyle='-', linewidth=1, color='black')

    # salvo i plot su file
    filename = 'plot/' + unique_name
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()
    
summarize_diagnostics(history)